Memento - Visualization of chunk information 

In [5]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('/workspaces/memento-mc/lab/data/memento_world_snapshot.csv')

# absolute chunk coordinates
df['x'] = df['regionx'] * 32 + df['chunkx']
df['z'] = df['regionz'] * 32 + df['chunkz']

# normalize inhabitance
df['inhabitance'] = (
    df['timeinhabited_ticks'] - df['timeinhabited_ticks'].min()
) / (
    df['timeinhabited_ticks'].max() - df['timeinhabited_ticks'].min()
)

# pivot to 2D grid
grid = df.pivot(index='z', columns='x', values='inhabitance')


In [6]:
fig = px.imshow(
    grid,
    origin='lower',
    color_continuous_scale='YlOrRd',
    title='Minecraft Chunk Inhabitance Map',
    labels=dict(color='Inhabitance (normalized)')
)

fig.update_layout(
    xaxis_title='Chunk X',
    yaxis_title='Chunk Z',
    yaxis_scaleanchor='x'  # critical: square chunks
)

fig.show()


In [8]:
# --- Add full region borders + emphasize origin (0,0) ---

REGION_SIZE = 32

# raw grid extents (chunk coords)
x_min_raw, x_max_raw = grid.columns.min(), grid.columns.max()
z_min_raw, z_max_raw = grid.index.min(), grid.index.max()

# expand to full region-aligned bounds
x_min = (x_min_raw // REGION_SIZE) * REGION_SIZE
x_max = ((x_max_raw // REGION_SIZE) + 1) * REGION_SIZE

z_min = (z_min_raw // REGION_SIZE) * REGION_SIZE
z_max = ((z_max_raw // REGION_SIZE) + 1) * REGION_SIZE

# helper to generate region boundaries
def region_lines(min_val, max_val, size):
    return range(int(min_val), int(max_val) + 1, size)

# vertical region borders (X)
for x in region_lines(x_min, x_max, REGION_SIZE):
    fig.add_shape(
        type="line",
        x0=x, x1=x,
        y0=z_min, y1=z_max,
        line=dict(color="rgba(0,0,0,0.35)", width=1),
        layer="above"
    )

# horizontal region borders (Z)
for z in region_lines(z_min, z_max, REGION_SIZE):
    fig.add_shape(
        type="line",
        x0=x_min, x1=x_max,
        y0=z, y1=z,
        line=dict(color="rgba(0,0,0,0.35)", width=1),
        layer="above"
    )

# --- Emphasize origin axes (0,0) ---

fig.add_shape(
    type="line",
    x0=0, x1=0,
    y0=z_min, y1=z_max,
    line=dict(color="black", width=2),
    layer="above"
)

fig.add_shape(
    type="line",
    x0=x_min, x1=x_max,
    y0=0, y1=0,
    line=dict(color="black", width=2),
    layer="above"
)

fig

